# Does this open?!?!?!?!

In [ ]:
from datetime import timedelta
import librosa
import numpy as np
import os
import pandas as pd
from scipy.signal import butter, filtfilt
from scipy.special import expit
import tensorflow as tf
import time
from tqdm import tqdm_notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
os.chdir('/content/drive/MyDrive/OVEN-test/')

current_directory = os.getcwd()

print("Current Directory:", current_directory)

In [ ]:
# Path to folder of recordings for analysis, uncomment the appropriate line to define the directory holding your recording files

# If running on a local machine then modify and use the following
#RECORDINGS = 'C:/file/path/here/'

# If running on Google Colan then modify and use the following
RECORDINGS = '/content/drive/MyDrive/OVEN-test/Recordings_OVEN-test/'
#RECORDINGS = '/content/drive/MyDrive/file/recording/path/'

In [ ]:
os.listdir(RECORDINGS)

In [ ]:
# Test code!!!!!!!

def highpass_filter(y, fc, sr):
    nyq = 0.5 * sr
    b, a = butter(4, fc/nyq, 'high')
    return filtfilt(b, a, y)

def normalise_audio(y, sr=16000):
    y = highpass_filter(y=y, fc=500.0, sr=sr)
    y -= np.mean(y)
    return y

def spectrogram_from_audio(y, sr):
    sr = 16000
    n_fft = 128
    hop_length = 64
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=np.blackman)[10:,:] # remove DC
    D = librosa.power_to_db(np.abs(D)**2, ref=1.0)
    D = np.swapaxes(D,0,1)[np.newaxis,np.newaxis,:,:] # [batch,height,width,channels]
    return D

def process_directory(recordings):
    # make sure there is somewhere to save results
    results_location = os.path.join(recordings, 'results')
    if not os.path.exists(results_location):
        os.makedirs(results_location)

    # find tensorflow model files
    meta_graph_path = 'oven-model-22022017.meta'
    checkpoint_path = 'oven-model-22022017'

    # list all audio files in directory
    audio_files = librosa.util.find_files(recordings, ext=['flac', 'wav'], recurse=True, case_sensitive=False, limit=None, offset=0)
    audio_files = list(set(audio_files)) # remove duplicates - not sure why find_files produces them!

    tf.compat.v1.reset_default_graph()
    with tf.compat.v1.Session() as sess:
        new_saver = tf.compat.v1.train.import_meta_graph(meta_graph_path)
        new_saver.restore(sess, checkpoint_path)
        features = tf.compat.v1.get_collection('input')[0]
        is_training = tf.compat.v1.get_collection('is_training')[0]
        level = tf.compat.v1.get_collection('level')[0]
        detection_fn = tf.compat.v1.get_collection('detection_fn')[0]

        for filename in tqdm_notebook(audio_files):
            y, sr = librosa.load(filename, sr=16000, mono=True)
            y = normalise_audio(y)
            S = spectrogram_from_audio(y, sr)
            level_dB, detection_function = sess.run([level, detection_fn], feed_dict={features: S, is_training: False})
            level_dB = level_dB[0,0,:,0]
            detection_function = detection_function[0,0,:,0]
            detection_function = expit(detection_function)
            duration_secs = y.size / float(sr)
            times = np.interp(np.arange(detection_function.size), [0,detection_function.size-1], [0,duration_secs])
            df = pd.DataFrame({'0': times, '1': detection_function, '2': level_dB})

            base = os.path.basename(filename)
            filename = os.path.splitext(base)[0]
            savename = os.path.join(results_location, filename + '_detection_function.csv')
            df.to_csv(savename, header=False, index=False)

process_directory(RECORDINGS)